In [ ]:
# default_exp recalibration

# Recalibration

> Functions related to recalibrating

This notebook contains everything to perform a recalibration.

Current ToDo here:

- Write introduction / comment more on functions
- Describe approach
- Cite relevant papers

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from scipy.interpolate import griddata
import pandas as pd
import numpy as np
import os
from alphapept.score import score_x_tandem
import alphapept.io
from multiprocessing import Pool


def estimate_stepsize(df, min_mz_step = 80, min_rt_step = 50, **kwargs):
    """
    Function to estimate the stepsize for recalibration

    """
    points_rt = np.sort(df['rt'].values)[::min_rt_step]
    points_mz = np.sort(df['mz'].values)[::min_mz_step]

    mz_step = np.round(np.mean(np.diff(points_mz))) #1 Th precision
    rt_step = np.round(np.mean(np.diff(points_rt)), 1) #0.1 minute precision

    # Minimum values are 1 Da and 0.1 minutes
    if mz_step < 1:
        mz_step = 1
    if rt_step < 0.1:
        rt_step = 0.1

    return mz_step, rt_step


def get_calibration(df, features, minimum_score = 20, outlier_std = 3, method='linear', min_mz_step=80, min_rt_step=50, callback = None, **kwargs):
    """
    Calibration

    Brief description

    (1) From the scored psms isolate the o_mass ppm offset. Filter for x std deviations
    (2) Estimate a stepsize for interpolation to ensure minimum sampling
    (3) Use points to interpolate on a grid
    (4) Use grid to interpolate offset for features

    """

    # Remove outliers for calibration
    o_mass_std = np.abs(df['o_mass_ppm'].std())
    o_mass_mean = df['o_mass_ppm'].mean()

    df_sub = df.query('score > @minimum_score and o_mass_ppm < @o_mass_mean+@outlier_std*@o_mass_std and o_mass_ppm > @o_mass_mean-@outlier_std*@o_mass_std').copy()

    if len(df_sub) > np.max([min_mz_step, min_rt_step]): #need a minimum number of points to estimate stepsize

        mz_step, rt_step = estimate_stepsize(df_sub, min_mz_step, min_rt_step, **kwargs)

        #Define a grid for valid interpolation

        min_x = np.floor(df_sub['mz'].min())
        max_x = np.ceil(df_sub['mz'].max())
        step_x = mz_step

        min_y = np.floor(df_sub['rt'].min())
        max_y = np.ceil(df_sub['rt'].max())
        step_y = rt_step

        tx = np.arange(min_x, max_x, step_x) #mz
        ty = np.arange(min_y, max_y, step_y) #rt

        xx, yy = np.meshgrid(tx, ty)

        f1 = griddata(
            df_sub[['mz','rt']].values,
            df_sub['o_mass_ppm'].values,
            (xx, yy),
            fill_value=0,
            method=method,
            rescale=True
        )

        f2 = griddata(
            np.vstack([xx.flatten(),yy.flatten()]).T,
            f1.flatten(),
            df_sub[['mz','rt']].values,
            fill_value=0,
            method=method,
            rescale=True
        )

        df_sub['o_mass_ppm_offset'] = f2
        df_sub['o_mass_ppm_calib'] = (df_sub['o_mass_ppm']-df_sub['o_mass_ppm_offset'])

        #Apply to features

        offset = griddata(
            np.vstack([xx.flatten(), yy.flatten()]).T,
            f1.flatten(),
            features[['mz_matched','rt_matched']].values,
            fill_value=0,
            method = method,
            rescale=True
        ) / 1e6*features['mass_matched']

    else:
        offset = 0

        df_sub['o_mass_ppm_offset'] = 0
        df_sub['o_mass_ppm_calib'] = (df_sub['o_mass_ppm']-df_sub['o_mass_ppm_offset'])

    corrected_mass = features['mass_matched'] - offset

    o_mass_ppm_mean = df_sub['o_mass_ppm_calib'].mean()
    o_mass_ppm_std = df_sub['o_mass_ppm_calib'].std()

    if np.isnan(o_mass_ppm_std):
        o_mass_ppm_std = 0

    return corrected_mass, o_mass_ppm_std


def calibrate_hdf(to_process):

    # TODO Only features are calibrated, not raw MS1 signals.
    # What if features are not present?
    
    path, settings = to_process

    ms_file = alphapept.io.MS_Data_File(path, is_overwritable=True)
    
    features = ms_file.read(dataset_name='features')

    try:
        psms =  ms_file.read(dataset_name='first_search')
    except KeyError: #no elements in search
        psms = pd.DataFrame()

    if len(psms) > 0 :
        df = score_x_tandem(
            psms,
            fdr_level=settings["search"]["peptide_fdr"],
            plot=False,
            verbose=False,
            **settings["search"]
        )
        corrected_mass, o_mass_ppm_std = get_calibration(
            df,
            features,
            **settings["calibration"]
        )
        ms_file.write(
            corrected_mass,
            dataset_name="corrected_mass",
            group_name="features"
        )
    else:
        o_mass_ppm_std = 0
    ms_file.write(
        o_mass_ppm_std,
        dataset_name="corrected_mass",
        group_name="features",
        attr_name="estimated_max_precursor_ppm"
    )
    return (path, o_mass_ppm_std)

def calibrate_hdf_parallel(settings, callback=None):

    ms_files = []

    for _ in settings['experiment']['file_paths']:
        base, ext = os.path.splitext(_)
        ms_files.append(base + '.ms_data.hdf')

    to_process = [(ms_file, settings) for ms_file in ms_files]

    calibration_dict = {}

    n_processes = settings['general']['n_processes']

    with Pool(n_processes) as p:
        max_ = len(to_process)
        for i, _ in enumerate(p.imap_unordered(calibrate_hdf, to_process)):
            path, offset = _
            calibration_dict[path] = float(offset)
            if callback:
                callback((i+1)/max_)


    return [calibration_dict[_]*settings['search']['calibration_std'] for _ in ms_files]

Calbration with regards to database

In [ ]:
#export

import alphapept.io
import pandas as pd
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
import numba
import scipy.signal
import scipy.interpolate
import alphapept.fasta

def get_db_targets(
    db_file_name,
    max_ppm=100,
    min_distance=0.5,
    ms_level=2,
):
    if ms_level == 1:
        db_mzs_ = alphapept.fasta.read_database(db_file_name, 'precursors')
    elif ms_level == 2:
        db_mzs_ = alphapept.fasta.read_database(db_file_name, 'fragmasses')
    else:
        raise ValueError(f"{ms_level} is not a valid ms level")
    tmp_result = np.bincount(
        (
            np.log10(
                db_mzs_[
                    np.isfinite(db_mzs_) & (db_mzs_ > 0)
                ].flatten()
            ) * 10**6
        ).astype(np.int64)
    )
    db_mz_distribution = np.zeros_like(tmp_result)
    for i in range(1, max_ppm):
        db_mz_distribution[i:] += tmp_result[:-i]
        db_mz_distribution[:-i] += tmp_result[i:]
    peaks = scipy.signal.find_peaks(db_mz_distribution, distance=max_ppm)[0]
    db_targets = 10 ** (peaks / 10**6)
#     db_vals = db_mz_distribution[peaks]
#     plt.vlines(db_targets, 0, db_vals)
    db_array = np.zeros(int(db_targets[-1]) + 1, dtype=np.float64)
    last_int_mz = -1
    last_mz = -1
    for mz in db_targets:
        mz_int = int(mz)
        if (mz_int != last_int_mz) & (mz > (last_mz + min_distance)):
            db_array[mz_int] = mz
        else:
            db_array[mz_int] = 0
        last_int_mz = mz_int
        last_mz = mz
    return db_array

In [ ]:
#export

def align_run_to_db(
    ms_data_file_name,
    db_array,
    max_ppm_distance=1000000,
    rt_step_size=0.1,
    plot_ppms=False,
    ms_level=2,
):
    ms_data = alphapept.io.MS_Data_File(ms_data_file_name)
    if ms_level == 1:
        mzs = ms_data.read(dataset_name="mass_matched", group_name="features")
        rts = ms_data.read(dataset_name="rt_matched", group_name="features")
    elif ms_level == 2:
        mzs = ms_data.read(dataset_name="Raw/MS2_scans/mass_list_ms2")
        inds = ms_data.read(dataset_name="Raw/MS2_scans/indices_ms2")
        precursor_rts = ms_data.read(dataset_name="Raw/MS2_scans/rt_list_ms2")
        rts = np.repeat(precursor_rts, np.diff(inds))
    else:
        raise ValueError(f"{ms_level} is not a valid ms level")
    
    selected = mzs.astype(np.int64)
    ds = np.zeros((3, len(selected)))
    if len(db_array) < len(selected) + 1:
        tmp = np.zeros(len(selected) + 1)
        tmp[:len(db_array)] = db_array
        db_array = tmp
    ds[0] = mzs - db_array[selected - 1]
    ds[1] = mzs - db_array[selected]
    ds[2] = mzs - db_array[selected + 1]
    min_ds = np.take_along_axis(
        ds,
        np.expand_dims(np.argmin(np.abs(ds), axis=0), axis=0),
        axis=0
    ).squeeze(axis=0)
    ppm_ds = min_ds / mzs * 10**6

    selected = np.abs(ppm_ds) < max_ppm_distance
    selected &= np.isfinite(rts)
    rt_order = np.argsort(rts)
    rt_order = rt_order[selected[rt_order]]

    
    ordered_rt = rts[rt_order]
    ordered_ppm = ppm_ds[rt_order]
    
    rt_idx_break = np.searchsorted(
        ordered_rt,
        np.arange(ordered_rt[0], ordered_rt[-1], rt_step_size),
        "left"
    )
    median_ppms = np.empty(len(rt_idx_break) - 1)
    for i in range(len(median_ppms)):
        median_ppms[i] = np.median(
            ordered_ppm[rt_idx_break[i]: rt_idx_break[i + 1]]
        )

    if plot_ppms:
        import matplotlib.pyplot as plt
        plt.plot(
            rt_step_size + np.arange(
                ordered_rt[0],
                ordered_rt[-1],
                rt_step_size
            )[:-1],
            median_ppms
        )
        plt.show()
        
    estimated_errors = scipy.interpolate.griddata(
        rt_step_size / 2 + np.arange(
            ordered_rt[0],
            ordered_rt[-1] - 2 * rt_step_size,
            rt_step_size
        ),
        median_ppms,
        rts,
        fill_value=0,
        method="linear",
        rescale=True
    )
    
    estimated_errors[~np.isfinite(estimated_errors)] = 0
        
    return estimated_errors

In [ ]:
#export

def calibrate_fragments(
    db_file_name,
    ms_data_file_name,
    ms_level=2
):
    db_array = get_db_targets(
        db_file_name,
        max_ppm=100,
        min_distance=0.5,
        ms_level=ms_level,
    )
    estimated_errors = align_run_to_db(
        ms_data_file_name,
        db_array=db_array,
        ms_level=ms_level,
        plot_ppms=False,
    )
    
    ms_file = alphapept.io.MS_Data_File(ms_data_file_name, is_overwritable=True)
    ms_file.write(
        estimated_errors,
        dataset_name="corrected_fragment_mzs",
    )

In [ ]:
#hide
from nbdev.export import *
notebook2script()